In [ ]:
print('Starting NFL Fantasy Pipeline...')
print('Pipeline complete!')

In [64]:
#=========================== EXTRACTION ===========================
# sample data

players = [
    {
        'name': 'Josh Allen'
        ,'position': 'QB'
        ,'passing_yards': 300
        ,'passing_touchdowns': 2
        ,'rushing_yards': 50
        ,'rushing_touchdowns': 1
        ,'receptions': 0
        ,'receiving_yards': 0
        ,'receiving_touchdowns': 0
    },
    {
        'name': 'Derrick Henry'
        ,'position': 'RB'
        ,'passing_yards': 0
        ,'passing_touchdowns': 0
        ,'rushing_yards': 120
        ,'rushing_touchdowns': 2
        ,'receptions': 3
        ,'receiving_yards': 25
        ,'receiving_touchdowns': 0
    },
    {
        'name': 'Davante Adams'
        ,'position': 'WR'
        ,'passing_yards': 0
        ,'passing_touchdowns': 0
        ,'rushing_yards': 0
        ,'rushing_touchdowns': 0
        ,'receptions': 8
        ,'receiving_yards': 95
        ,'receiving_touchdowns': 1
    }
]

# print(f'Our sample player: {player}')

In [65]:
#=========================== TRANSFORM ===========================
# calculate fantasy points

def calculate_fantasy_points(player): 
    # Passing yards: 1 point per 25 yards (so 300 yards = 12 points)
    # Passing touchdowns: 4 points each
    # Rushing yards: 1 point per 10 yards
    # Rushing touchdowns: 6 points each
    # Receptions: 1 point each
    # Receiving Yards: 1 point per 10 yards
    # Receiving TDs: 6 points each

    passing_points = player['passing_yards'] / 25 * 1
    passing_td_points = player['passing_touchdowns'] * 4
    rushing_points = player['rushing_yards'] / 10 * 1
    rushing_td_points = player['rushing_touchdowns'] * 6
    reception_points = player['receptions'] * 1
    receiving_points = player['receiving_yards'] / 10 * 1
    receiving_td_points = player['receiving_touchdowns'] * 6

    total_points = (passing_points 
                    + passing_td_points 
                    + rushing_points 
                    + rushing_td_points
                    + reception_points
                    + receiving_points 
                    + receiving_td_points
    )

    return total_points

print(f'We now have {len(players)} players')
print('Players:', [p['name'] for p in players])

We now have 3 players
Players: ['Josh Allen', 'Derrick Henry', 'Davante Adams']


In [66]:
# calculate for all players in dataset

# iterate through every player in the dataset to calculate their total fantasy points
for player in players:
    points = calculate_fantasy_points(player)
    print(f'{player['name']} ({player['position']}): {points} fantasy points')

Josh Allen (QB): 31.0 fantasy points
Derrick Henry (RB): 29.5 fantasy points
Davante Adams (WR): 23.5 fantasy points


In [67]:
#=========================== LOAD ===========================

import sqlite3

# connect to database
conn = sqlite3.connect('nfl_fantasy.db')
print('Connected to database!')
cursor = conn.cursor()

#helps when re-running the code
cursor.execute('''drop table if exists player_stats''')

# create table for player/stats
cursor.execute('''
               create table player_stats (
                name text
                ,position text
                ,week integer
                ,fantasy_points real
                ,primary key (name, week)
                )
''')

conn.commit()
conn.close()
print('Table created!')

Connected to database!
Table created!


In [68]:
# insert player data 
conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

current_week = 1

for player in players:
    points = calculate_fantasy_points(player)
    cursor.execute('''
                insert or replace into player_stats 
                   (name
                   ,position
                   ,week
                   ,fantasy_points)
                values 
                   (?
                   ,?
                   ,?
                   ,?)
                ''', (player['name'], player['position'], current_week, points))
    
conn.commit()
conn.close()
print(f'Week {current_week} data inserted!')

Week 1 data inserted!


In [69]:
# read sql table to see if data made it in the db

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

# pull all data from table
cursor.execute('select * from player_stats')
results = cursor.fetchall()

conn.close()

print('Data in database:')
for i in results:
    print(i)

Data in database:
('Josh Allen', 'QB', 1, 31.0)
('Derrick Henry', 'RB', 1, 29.5)
('Davante Adams', 'WR', 1, 23.5)


In [ ]:
###################### Adding week 2 data to test if system can handle multiple weeks for the same players

In [ ]:
#=========================== extract ===========================

# Week 2 data - Josh Allen has a bad game, others do well
week2_players = [
    {
        "name": "Josh Allen"
        ,"position": "QB"
        ,"passing_yards": 200  # Bad passing game
        ,"passing_touchdowns": 1
        ,"rushing_yards": 20   # Less rushing
        ,"rushing_touchdowns": 0
        ,"receptions": 0
        ,"receiving_yards": 0
        ,"receiving_touchdowns": 0
    },
    {
        "name": "Derrick Henry"
        ,"position": "RB"
        ,"passing_yards": 0
        ,"passing_touchdowns": 0
        ,"rushing_yards": 180  # Big game!
        ,"rushing_touchdowns": 3
        ,"receptions": 2
        ,"receiving_yards": 30
        ,"receiving_touchdowns": 0
    },
    {
        "name": "Davante Adams"
        ,"position": "WR"
        ,"passing_yards": 0
        ,"passing_touchdowns": 0
        ,"rushing_yards": 0
        ,"rushing_touchdowns": 0
        ,"receptions": 12  # Monster receiving game
        ,"receiving_yards": 150
        ,"receiving_touchdowns": 2
    }
]

print("Week 2 player data created!")

Week 2 player data created!


In [ ]:
#=========================== transform ===========================

for player in week2_players:
    points = calculate_fantasy_points(player)
    print(f'{player['name']} ({player['position']}): {points} fantasy points')

Josh Allen (QB): 14.0 fantasy points
Derrick Henry (RB): 41.0 fantasy points
Davante Adams (WR): 39.0 fantasy points


In [72]:
#=========================== load ===========================

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

# insert week 2 data
current_week = 2

for player in week2_players:
    points = calculate_fantasy_points(player)
    cursor.execute('''
                insert or replace into player_stats 
                   (name
                   ,position
                   ,week
                   ,fantasy_points)
                values 
                   (?
                   ,?
                   ,?
                   ,?)
                ''', (player['name'], player['position'], current_week, points))
    
conn.commit()
conn.close()
print(f'Week {current_week} data inserted!')

Week 2 data inserted!


In [73]:
# read sql table to see if week 2 data made it in the db

conn = sqlite3.connect('nfl_fantasy.db')
cursor = conn.cursor()

cursor.execute('''select * from player_stats''')
results = cursor.fetchall()

conn.close()

print('All data in db:')
for i in results:
    print(i)

All data in db:
('Josh Allen', 'QB', 1, 31.0)
('Derrick Henry', 'RB', 1, 29.5)
('Davante Adams', 'WR', 1, 23.5)
('Josh Allen', 'QB', 2, 14.0)
('Derrick Henry', 'RB', 2, 41.0)
('Davante Adams', 'WR', 2, 39.0)


In [ ]:
#=========================== Try pulling in real data ===========================

In [ ]:
import kagglehub
import shutil
import os

# download fantasy-football-agent-dataset from kaggle

# url: https://www.kaggle.com/datasets/sidsugathan/fantasy-football-agent-dataset

try:
    path = kagglehub.dataset_download("sidsugathan/fantasy-football-agent-dataset")
    print("Path to dataset files:", path)
    print("Dataset downloaded successfully!")

    # copy downloaded files to current project folder
    project_data_path = './data/'
    os.makedirs(project_data_path, exist_ok=True)

    # copy all csv files to project folder
    for file in os.listdir(path):
        if file.endswith('.csv'):
            source = os.path.join(path, file)
            destination = os.path.join(project_data_path, file)
            shutil.copy2(source, destination)   # .copy2() overwrites files that already exist, avoids dupes
            print(f'Copied {file} to ./data/')

    print(f'\nFiles now in your project at: {project_data_path}')

except Exception as e:
    print(f"Error downloading dataset: {e}")
    print("This might require Kaggle authentication")

Path to dataset files: C:\Users\kcmcs\.cache\kagglehub\datasets\sidsugathan\fantasy-football-agent-dataset\versions\16
Dataset downloaded successfully!
Copied nfl_coaching_staff_2021_2024.csv to ./data/
Copied performance_data_with_ids_cleaned_2021_2024.csv to ./data/
Copied player_id_mapping_master_cleaned.csv to ./data/
Copied pre_draft_rankings_with_2024_tiers.csv to ./data/
Copied rookie_metadata_with_blurbs_and_ids_cleaned_2021_2024.csv to ./data/
Copied weekly_fantasy_scores_2024_with_ids.csv to ./data/

Files now in your project at: ./data/
